# Introduction
Sadly, suicide is a common thing all around the world. We'll be taking a small look on suicide factors starting with gender, then age. Later we'll be taking a suicide rate among countries with an amount of research of key factors. Lets begin.

### Index
<font color = 'blue'>

1. [Load and Check Data](#1)
2. [Factor Analysis](#2)
    * [Gender](#3)
    * [Age](#4)
    * [Suicide Rate](#5)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import seaborn as sns

<a id = "1"></a><dr>
## Load and Check Data

In [ ]:
#Reading the data.
df = pd.read_csv('../input/suicide-rates-overview-1985-to-2016/master.csv')

In [ ]:
df.head(10)

In [ ]:
#renaming some columns in appropriate form.
df.rename(columns={"suicides/100k pop":"suicides_pop", "HDI for year": "HDI_for_year", "gdp_per_capita ($)":"gdp_per_capita",
                    " gdp_for_year ($) ":"gdp_for_year"}, inplace=True)
df

In [ ]:
#removing the commas.
df["gdp_for_year"] = df["gdp_for_year"].str.replace(",","").astype(np.int64)
df["gdp_for_year"]

In [ ]:
df["age"] = df["age"].str.replace("5-14 years","05-14 years")
df["age"]

<a id = "2"></a><dr>
## Factor Analysis

<a id = "3"></a><dr>
### Gender

In [ ]:
#Making a lineplot graph in order to see the relation of gender.
sns.set(style="darkgrid")
sns.lineplot(x = df[df.sex == "male"].year, y = df.suicides_no, ci = None, color='blue')
sns.lineplot(x = df[df.sex == "female"].year, y = df.suicides_no, ci = None, color= 'red')
plt.legend(["male","female"])
plt.show()

Males were more likely to commit suicide. After the WW2, maybe because of psychological issues males committed suicide.

<a id = "4"></a><dr>
### Age

In [ ]:
df1 = df.groupby(["year","age"])["suicides_no", "population"].sum().reset_index() 
#making a group on year by age, taking the sum of suicides_no and population.
plt.figure(figsize=(9,7))
sns.lineplot(x = "year", y = df1.suicides_no*100/df1.population, hue = "age",
             data = df1, linewidth = 3, style = "age", markers=True, dashes=False)
plt.xlabel('year')
plt.ylabel('ratio')
plt.xticks(rotation = 90)
plt.show()

Just before 90s, suicides are dramatically increasing between age of 15+

<a id = "5"></a><dr>
### Suicide Rate

In [ ]:
#Calculating suicide rate
df_sum_all = df.groupby(["country","year"])["suicides_no"].sum().sort_index(ascending=True)[:] * 100
df_population_sum = df.groupby(["country","year"]).population.sum().sort_index(ascending=False)[:]
df_total = df_sum_all/df_population_sum
df_total.head(10)

In [ ]:
country_dict={}

for country in df_total.index.get_level_values(0):
    if country not in country_dict.keys():
        country_dict[country] = df_total[country].mean() #assigning the mean of other countries if a county is not in keys.
    else:
        pass

list1 = list(country_dict.items()) #assigning a list for items
list1.sort(key= lambda pair:pair[1], reverse = True) #using the key function here to have an order starting alphabetically descending.

country_list = [a[0] for a in list1] #countries in indexes
country_suicide = [a[1] for a in list1] #suicides in columns

In [ ]:
#Making a barplot of suicide rate versus country
plt.figure(figsize=(8,32))
sns.barplot(x=country_suicide[:],y=country_list[:], palette="flare")
plt.xlabel("ratio of suicide")
plt.ylabel("country")
plt.title("suicide rate versus country")
plt.show()

In [ ]:
df_gdp = df.groupby(["country","year"]).gdp_per_capita.mean() #taking the mean of gdp
corr_eff = {}
for country in country_list[:10]: #Taking the first 10 countries.
    slope, intercept, r_value, p_value, std_err = stats.linregress(df_gdp[country].values,df_total[country].values)
    corr_eff[country] = float(r_value)
    
sns.barplot(x=list(corr_eff.keys()), y=list(corr_eff.values()), palette = "rocket")
plt.xticks(rotation = 90)
plt.xlabel("Country")
plt.ylabel("correlation coefficient")
plt.title("GDP versus suicides")
plt.show()